In [10]:
!pip install wbgapi

  Obtaining dependency information for wbgapi from https://files.pythonhosted.org/packages/00/12/224030af4886e119a3d03b709f7130e9601a4d15332e1d6a35671b25a4de/wbgapi-1.0.12-py3-none-any.whl.metadata


In [1]:
import sys
import subprocess

# Szükséges csomagok ellenőrzése és telepítése
def install(package):
    try:
        __import__(package)
    except ImportError:
        subprocess.check_call([sys.executable, "-m", "pip", "install", package])

install('pandas_datareader')
install('yfinance')

import pandas_datareader.data as web
import yfinance as yf
import pandas as pd
import requests
import time
import numpy as np

# --- 1. BEÁLLÍTÁSOK ---
START_DATE = "2021-01-01"
END_DATE = "2024-01-01"
EVENT_DATE = pd.Timestamp("2022-02-24")

# --- 2. TELJES ORSZÁGLISTA (47 ország a PDF alapján) ---
countries = {
    # Stooq hibrid megoldások (Közép-Európa)
    'Hungary':      {'ticker': 'BUX.BD',    'stooq_ticker': '^BUX', 'currency': 'HUF=X',    'iso': 'HU', 'NATO': 1, 'DEV': 0},
    'Poland':       {'ticker': '^WIG',      'stooq_ticker': 'WIG',  'currency': 'PLN=X',    'iso': 'PL', 'NATO': 1, 'DEV': 0},
    'Czech Republic':{'ticker': '^PX',      'stooq_ticker': '^PX',  'currency': 'CZK=X',    'iso': 'CZ', 'NATO': 1, 'DEV': 0},
    
    # Fejlett piacok (Developed - 23 db)
    'US':           {'ticker': '^DJI',       'iso': 'US', 'currency': 'USD',      'NATO': 1, 'DEV': 1},
    'Spain':        {'ticker': '^IBEX',      'iso': 'ES', 'currency': 'EURUSD=X', 'NATO': 1, 'DEV': 1},
    'Italy':        {'ticker': 'FTSEMIB.MI','iso': 'IT', 'currency': 'EURUSD=X', 'NATO': 1, 'DEV': 1},
    'Germany':      {'ticker': '^GDAXI',    'iso': 'DE', 'currency': 'EURUSD=X', 'NATO': 1, 'DEV': 1},
    'France':       {'ticker': '^FCHI',     'iso': 'FR', 'currency': 'EURUSD=X', 'NATO': 1, 'DEV': 1},
    'UK':           {'ticker': '^FTSE',     'iso': 'GB', 'currency': 'GBPUSD=X', 'NATO': 1, 'DEV': 1},
    'Belgium':      {'ticker': '^BFX',      'iso': 'BE', 'currency': 'EURUSD=X', 'NATO': 1, 'DEV': 1},
    'Switzerland':  {'ticker': '^SSMI',     'iso': 'CH', 'currency': 'CHFUSD=X', 'NATO': 0, 'DEV': 1},
    'Netherlands':  {'ticker': '^AEX',      'iso': 'NL', 'currency': 'EURUSD=X', 'NATO': 1, 'DEV': 1},
    'Canada':       {'ticker': '^GSPTSE',   'iso': 'CA', 'currency': 'CAD=X',    'NATO': 1, 'DEV': 1},
    'Portugal':     {'ticker': 'PSI20.LS',  'iso': 'PT', 'currency': 'EURUSD=X', 'NATO': 1, 'DEV': 1},
    'Austria':      {'ticker': '^ATX',      'iso': 'AT', 'currency': 'EURUSD=X', 'NATO': 0, 'DEV': 1},
    'Israel':       {'ticker': '^TA125.TA', 'iso': 'IL', 'currency': 'ILS=X',    'NATO': 0, 'DEV': 1},
    'Sweden':       {'ticker': '^OMX',      'iso': 'SE', 'currency': 'SEK=X',    'NATO': 0, 'DEV': 1},
    'Ireland':      {'ticker': '^ISEQ',     'iso': 'IE', 'currency': 'EURUSD=X', 'NATO': 0, 'DEV': 1},
    'Norway':       {'ticker': 'OSEBX.OL',  'iso': 'NO', 'currency': 'NOK=X',    'NATO': 1, 'DEV': 1},
    'Australia':    {'ticker': '^AXJO',     'iso': 'AU', 'currency': 'AUDUSD=X', 'NATO': 0, 'DEV': 1},
    'Denmark':      {'ticker': '^OMXC20',   'iso': 'DK', 'currency': 'DKK=X',    'NATO': 1, 'DEV': 1},
    'Japan':        {'ticker': '^N225',     'iso': 'JP', 'currency': 'JPY=X',    'NATO': 0, 'DEV': 1},
    'Finland':      {'ticker': '^OMXH25',   'iso': 'FI', 'currency': 'EURUSD=X', 'NATO': 0, 'DEV': 1},
    'Singapore':    {'ticker': '^STI',      'iso': 'SG', 'currency': 'SGD=X',    'NATO': 0, 'DEV': 1},
    'Hong Kong':    {'ticker': '^HSI',      'iso': 'HK', 'currency': 'HKD=X',    'NATO': 0, 'DEV': 1},
    'New Zealand':  {'ticker': '^NZ50',     'iso': 'NZ', 'currency': 'NZDUSD=X', 'NATO': 0, 'DEV': 1},

    # Feltörekvő piacok (Emerging - 24 db)
    'China':        {'ticker': '000001.SS', 'iso': 'CN', 'currency': 'CNY=X',    'NATO': 0, 'DEV': 0},
    'Turkey':       {'ticker': 'XU100.IS',  'iso': 'TR', 'currency': 'TRY=X',    'NATO': 1, 'DEV': 0},
    'Brazil':       {'ticker': '^BVSP',     'iso': 'BR', 'currency': 'BRL=X',    'NATO': 0, 'DEV': 0},
    'South Korea':  {'ticker': '^KS11',     'iso': 'KR', 'currency': 'KRW=X',    'NATO': 0, 'DEV': 0},
    'India':        {'ticker': '^BSESN',    'iso': 'IN', 'currency': 'INR=X',    'NATO': 0, 'DEV': 0},
    'Malaysia':     {'ticker': '^KLSE',     'iso': 'MY', 'currency': 'MYR=X',    'NATO': 0, 'DEV': 0},
    'Philippines':  {'ticker': 'PSEI.PS',   'iso': 'PH', 'currency': 'PHP=X',    'NATO': 0, 'DEV': 0},
    'Mexico':       {'ticker': '^MXX',      'iso': 'MX', 'currency': 'MXN=X',    'NATO': 0, 'DEV': 0},
    'Indonesia':    {'ticker': '^JKSE',     'iso': 'ID', 'currency': 'IDR=X',    'NATO': 0, 'DEV': 0},
    'Saudi Arabia': {'ticker': '^TASI.SR',  'iso': 'SA', 'currency': 'SAR=X',    'NATO': 0, 'DEV': 0},
    'Thailand':     {'ticker': '^SET.BK',   'iso': 'TH', 'currency': 'THB=X',    'NATO': 0, 'DEV': 0},
    'South Africa': {'ticker': '^JN0U.JO', 'iso': 'ZA', 'currency': 'ZAR=X',    'NATO': 0, 'DEV': 0},
    'Taiwan':       {'ticker': '^TWII',     'iso': 'TW', 'currency': 'TWD=X',    'NATO': 0, 'DEV': 0},
    'Chile':        {'ticker': 'ECH',       'iso': 'CL', 'currency': 'CLP=X', 'NATO': 0, 'DEV': 0},
    'Peru':         {'ticker': 'EPU',       'iso': 'PE', 'currency': 'PEN=X', 'NATO': 0, 'DEV': 0},
    'Colombia':     {'ticker': 'GXG',       'iso': 'CO', 'currency': 'COP=X', 'NATO': 0, 'DEV': 0},
    'Greece':       {'ticker': 'GREK',      'iso': 'GR', 'currency': 'EURUSD=X', 'NATO': 1, 'DEV': 0},
    'UAE':          {'ticker': 'UAE',       'iso': 'AE', 'currency': 'AED=X', 'NATO': 0, 'DEV': 0},
    'Qatar':        {'ticker': 'QAT',       'iso': 'QA', 'currency': 'QAR=X', 'NATO': 0, 'DEV': 0},
    'Kuwait':       {'ticker': 'KWT',       'iso': 'KW', 'currency': 'KWD=X', 'NATO': 0, 'DEV': 0},
    'Egypt':        {'ticker': 'EGPT',      'iso': 'EG', 'currency': 'EGP=X', 'NATO': 0, 'DEV': 0},
}

# --- 3. ADATGYŰJTŐ FÜGGVÉNYEK ---
def get_wb_trade_data(iso_code, year=2021):
    url = f"http://api.worldbank.org/v2/country/{iso_code}/indicator/NE.TRD.GNFS.ZS?format=json&date={year}"
    try:
        response = requests.get(url, timeout=5)
        data = response.json()
        if len(data) > 1 and data[1] is not None:
            return data[1][0]['value']
    except: return None
    return None

def get_exrate_value(currency_ticker, event_date):
    if currency_ticker == 'USD': return 1.0
    try:
        start_dl = (event_date - pd.Timedelta(days=40)).strftime('%Y-%m-%d')
        end_dl = event_date.strftime('%Y-%m-%d')
        fx_data = yf.download(currency_ticker, start=start_dl, end=end_dl, progress=False)
        if fx_data.empty: return None
        val = fx_data['Close'].iloc[:, 0] if isinstance(fx_data.columns, pd.MultiIndex) else fx_data['Close']
        avg_rate = val.sort_index().tail(10).mean()
        if currency_ticker in ['EURUSD=X', 'GBPUSD=X', 'AUDUSD=X', 'NZDUSD=X', 'CHFUSD=X']:
             return 1.0 / avg_rate
        return avg_rate
    except: return None

# --- 4. FUTTATÁS ---
data_frames = []
exrate_cache, trade_cache = {}, {}

print(f"Adatgyűjtés indítása ({len(countries)} ország)...")

for country, info in countries.items():
    df_clean = None
    try:
        df = yf.download(info['ticker'], start=START_DATE, end=END_DATE, progress=False)
        if not df.empty:
            df_clean = pd.DataFrame(index=df.index)
            df_clean['Price'] = df['Close'].iloc[:, 0] if isinstance(df.columns, pd.MultiIndex) else df['Close']
    except: pass

    if (df_clean is None or df_clean.empty) and 'stooq_ticker' in info:
        try:
            df_stooq = web.DataReader(info['stooq_ticker'], 'stooq', start=START_DATE, end=END_DATE).sort_index()
            df_clean = pd.DataFrame(index=df_stooq.index, data={'Price': df_stooq['Close']})
        except: pass

    if df_clean is not None:
        df_clean = df_clean.reset_index()
        df_clean.rename(columns={df_clean.columns[0]: 'Date'}, inplace=True)
        df_clean['Date'] = pd.to_datetime(df_clean['Date'])
        df_clean['Return'] = df_clean['Price'].astype(float).pct_change()
        
        curr, iso = info['currency'], info['iso']
        if curr not in exrate_cache: exrate_cache[curr] = get_exrate_value(curr, EVENT_DATE)
        if iso not in trade_cache: trade_cache[iso] = get_wb_trade_data(iso)
        
        df_clean['Country'], df_clean['NATO'], df_clean['DEV'] = country, info['NATO'], info['DEV']
        df_clean['EXRATE'], df_clean['TRADE'] = exrate_cache[curr], trade_cache[iso]
        data_frames.append(df_clean)
        print(f"✅ {country}")

if data_frames:
    full_df = pd.concat(data_frames, ignore_index=True)
    # PAST számítása
    country_past = {c: full_df[(full_df['Country'] == c) & (full_df['Date'] < EVENT_DATE)].sort_values('Date').tail(20)['Return'].mean() for c in countries.keys()}
    full_df['PAST'] = full_df['Country'].map(country_past)
    
    full_df.dropna(subset=['Return']).to_csv("stock_market_final_panel.csv", index=False)
    print(f"\n🎉 KÉSZ! Fájl: stock_market_final_panel.csv ({len(full_df)} sor)")

Adatgyűjtés indítása (47 ország)...


C:\Users\andre\AppData\Local\Temp\ipykernel_28864\1947558614.py:116: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df = yf.download(info['ticker'], start=START_DATE, end=END_DATE, progress=False)
HTTP Error 404: {"quoteSummary":{"result":null,"error":{"code":"Not Found","description":"Quote not found for symbol: BUX.BD"}}}

1 Failed download:
['BUX.BD']: YFTzMissingError('possibly delisted; no timezone found')
C:\Users\andre\AppData\Local\Temp\ipykernel_28864\1947558614.py:98: FutureWarning: YF.download() has changed argument auto_adjust default to True
  fx_data = yf.download(currency_ticker, start=start_dl, end=end_dl, progress=False)


✅ Hungary


C:\Users\andre\AppData\Local\Temp\ipykernel_28864\1947558614.py:116: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df = yf.download(info['ticker'], start=START_DATE, end=END_DATE, progress=False)

1 Failed download:
['^WIG']: YFPricesMissingError('possibly delisted; no price data found  (1d 2021-01-01 -> 2024-01-01)')
C:\Users\andre\AppData\Local\Temp\ipykernel_28864\1947558614.py:116: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df = yf.download(info['ticker'], start=START_DATE, end=END_DATE, progress=False)

1 Failed download:
['^PX']: YFPricesMissingError('possibly delisted; no price data found  (1d 2021-01-01 -> 2024-01-01)')
C:\Users\andre\AppData\Local\Temp\ipykernel_28864\1947558614.py:98: FutureWarning: YF.download() has changed argument auto_adjust default to True
  fx_data = yf.download(currency_ticker, start=start_dl, end=end_dl, progress=False)


✅ Czech Republic


...

✅ Qatar


C:\Users\andre\AppData\Local\Temp\ipykernel_28864\1947558614.py:116: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df = yf.download(info['ticker'], start=START_DATE, end=END_DATE, progress=False)
C:\Users\andre\AppData\Local\Temp\ipykernel_28864\1947558614.py:98: FutureWarning: YF.download() has changed argument auto_adjust default to True
  fx_data = yf.download(currency_ticker, start=start_dl, end=end_dl, progress=False)


✅ Kuwait


C:\Users\andre\AppData\Local\Temp\ipykernel_28864\1947558614.py:116: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df = yf.download(info['ticker'], start=START_DATE, end=END_DATE, progress=False)
C:\Users\andre\AppData\Local\Temp\ipykernel_28864\1947558614.py:98: FutureWarning: YF.download() has changed argument auto_adjust default to True
  fx_data = yf.download(currency_ticker, start=start_dl, end=end_dl, progress=False)


✅ Egypt

🎉 KÉSZ! Fájl: stock_market_final_panel.csv (33736 sor)
